# Instalações

In [2]:
!pip install --quiet ultralytics
!pip install --quiet scikit-learn

from ultralytics import YOLO
import cv2
from collections import defaultdictimport numpy as npimport timefrom sklearn.cluster import KMeans
model = YOLO('yolov8n.pt')


# Detecção de pessoas e rastreamento


In [22]:
cap = cv2.VideoCapture("/home/aprigio/Downloads/Faculdade/Topicos PDI/Projeto TPDI/video1.mp4")

track_history = defaultdict(lambda: [])
person_id_counter = 0 
person_ids = {}  

seguir = True
deixar_rastro = True

frame_count = 0
frame_skip = 1 

# Iniciar o contador de tempo para o cálculo do FPS
prev_time = time.time()

resize_width = 640
resize_height = 360

while True:
    success, img = cap.read()

    if not success:
        break  # Sai do loop se não conseguir ler mais frames

    frame_count += 1
    if frame_count % frame_skip != 0:
        continue  # Pule este frame para melhorar o desempenho

    # Redimensiona o frame para resolução menor
    img = cv2.resize(img, (resize_width, resize_height))

    # YOLOv8 Processamento
    if seguir:
        results = model.track(img, persist=True)
    else:
        results = model(img)

    # Cria uma cópia da imagem original para desenhar apenas pessoas
    img_people_only = img.copy()

    # Processar a lista de resultados
    for result in results:
        if seguir and deixar_rastro:
            try:
                person_class_index = 0  # Índice da classe person na YOLOv8 é 0
                boxes = result.boxes.xywh.cpu()
                track_ids = result.boxes.id.int().cpu().tolist()
                class_ids = result.boxes.cls.int().cpu().tolist()

                # Desenha as caixas, IDs e rastros apenas para pessoas
                for box, track_id, class_id in zip(boxes, track_ids, class_ids):
                    if class_id == person_class_index:
                        x, y, w, h = map(int, box) 
                        
                        if track_id not in person_ids:
                            person_ids[track_id] = person_id_counter
                            person_id_counter += 1

                        person_id = person_ids[track_id]

                        # Desenha o retângulo ao redor das pessoas
                        cv2.rectangle(img_people_only, (x - w // 2, y - h // 2),
                                      (x + w // 2, y + h // 2), (0, 255, 0), 2)

                        # Adiciona o ID da pessoa na imagem perto do retângulo
                        cv2.putText(img_people_only, f'ID: {person_id}', (x - w // 2, y - h // 2 - 10),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

                        # Rastreia o histórico de posição para cada pessoa
                        track = track_history[track_id]
                        track.append((float(x), float(y)))  # Ponto central x, y
                        if len(track) > 30:  # Mantém no máximo 30 frames no histórico
                            track.pop(0)

                        # Desenha as linhas de rastreamento
                        points = np.array(track).astype(np.int32).reshape((-1, 1, 2))
                        cv2.polylines(img_people_only, [points], isClosed=False, color=(230, 0, 0), thickness=5)

            except Exception as e:
                print(f"Erro: {e}")
                pass

    # Calcula o FPS
    current_time = time.time()
    fps = 1 / (current_time - prev_time)
    prev_time = current_time

    # Desenha o FPS no canto da imagem
    cv2.putText(img_people_only, f'FPS: {int(fps)}', (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Mostra a imagem filtrada com apenas pessoas
    cv2.imshow("Frame", img_people_only)

    k = cv2.waitKey(1)
    if k == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
print("desligando")



0: 384x640 1 chair, 55.1ms
Speed: 2.0ms preprocess, 55.1ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 chair, 63.6ms
Speed: 1.4ms preprocess, 63.6ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 chair, 81.8ms
Speed: 1.0ms preprocess, 81.8ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 chair, 54.5ms
Speed: 1.6ms preprocess, 54.5ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 chairs, 51.3ms
Speed: 2.3ms preprocess, 51.3ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 chairs, 48.9ms
Speed: 1.9ms preprocess, 48.9ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 chairs, 48.7ms
Speed: 1.0ms preprocess, 48.7ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 chairs, 54.5ms
Speed: 1.5ms preprocess, 54.5ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 6

# Salvando as cores como caractéristicas 


In [35]:
cap = cv2.VideoCapture("/home/aprigio/Downloads/Faculdade/Topicos PDI/Projeto TPDI/video2.mp4")

track_history = defaultdict(lambda: [])
person_id_counter = 0  # Contador de IDs de pessoas
person_ids = {}  # Dicionário para mapear track_id para um person_id exclusivo

seguir = True
deixar_rastro = True

frame_count = 0
frame_skip = 2  # Ajuste o valor para pular mais frames e melhorar o desempenho

# Iniciar o contador de tempo para o cálculo do FPS
prev_time = time.time()

# Reduz a resolução para aumentar o FPS (opcional)
resize_width = 640
resize_height = 360

# Função para obter a cor predominante
def get_dominant_color(image, box):
    x, y, w, h = map(int, box)
    person_roi = image[y - h // 2: y + h // 2, x - w // 2: x + w // 2]

    if person_roi.size == 0:
        return (0, 0, 0)  # Evitar erro se a ROI estiver vazia

    # Redimensiona a imagem para facilitar o cálculo da cor dominante
    person_roi = cv2.resize(person_roi, (50, 50), interpolation=cv2.INTER_AREA)
    
    # Converte a imagem para um array 2D de pixels
    pixels = person_roi.reshape((-1, 3))
    
    # Usa o KMeans para encontrar a cor mais comum
    kmeans = KMeans(n_clusters=1)
    kmeans.fit(pixels)
    
    # A cor predominante será o centro do cluster
    dominant_color = kmeans.cluster_centers_[0].astype(int)
    
    return tuple(map(int, dominant_color))  # Certifique-se de retornar inteiros

while True:
    success, img = cap.read()

    if not success:
        break  # Sai do loop se não conseguir ler mais frames

    frame_count += 1
    if frame_count % frame_skip != 0:
        continue  # Pule este frame para melhorar o desempenho

    # Redimensiona o frame para resolução menor
    img = cv2.resize(img, (resize_width, resize_height))

    # YOLOv8 Processamento
    if seguir:
        results = model.track(img, persist=True)
    else:
        results = model(img)

    # Cria uma cópia da imagem original para desenhar apenas pessoas
    img_people_only = img.copy()

    # Processar a lista de resultados
    for result in results:
        if seguir and deixar_rastro:
            try:
                person_class_index = 0  # Índice da classe "person" na YOLOv8 é geralmente 0
                boxes = result.boxes.xywh.cpu()
                track_ids = result.boxes.id.int().cpu().tolist()
                class_ids = result.boxes.cls.int().cpu().tolist()

                # Desenha as caixas, IDs e rastros apenas para pessoas
                for box, track_id, class_id in zip(boxes, track_ids, class_ids):
                    if class_id == person_class_index:
                        x, y, w, h = map(int, box)  # Certifique-se de que as coordenadas são inteiras

                        # Atribui um novo person_id se ainda não estiver no dicionário
                        if track_id not in person_ids:
                            person_ids[track_id] = person_id_counter
                            person_id_counter += 1

                        person_id = person_ids[track_id]

                        # Obtém a cor predominante da pessoa
                        dominant_color = get_dominant_color(img, box)

                        # Desenha o retângulo ao redor das pessoas
                        cv2.rectangle(img_people_only, (x - w // 2, y - h // 2),
                                      (x + w // 2, y + h // 2), (0, 255, 0), 2)

                        # Adiciona o ID e a cor predominante na imagem perto do retângulo
                        cv2.putText(img_people_only, f'ID: {person_id}', (x - w // 2, y - h // 2 - 10),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
                        cv2.putText(img_people_only, f'Cor BGR {dominant_color}', (x - w // 2, y - h // 2 - 30),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, ((dominant_color[0]), int(dominant_color[1]), int(dominant_color[2])), 2)

                        # Rastreia o histórico de posição para cada pessoa
                        track = track_history[track_id]
                        track.append((float(x), float(y)))  # Ponto central x, y
                        if len(track) > 30:  # Mantém no máximo 30 frames no histórico
                            track.pop(0)

                        # Desenha as linhas de rastreamento
                        points = np.array(track).astype(np.int32).reshape((-1, 1, 2))
                        cv2.polylines(img_people_only, [points], isClosed=False, color=(230, 0, 0), thickness=5)

            except Exception as e:
                print(f"Erro: {e}")
                pass

    # Calcula o FPS
    current_time = time.time()
    fps = 1 / (current_time - prev_time)
    prev_time = current_time

    # Desenha o FPS no canto da imagem
    cv2.putText(img_people_only, f'FPS: {int(fps)}', (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Mostra a imagem filtrada com apenas pessoas
    cv2.imshow("Frame", img_people_only)

    k = cv2.waitKey(1)
    if k == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
print("desligando")



0: 384x640 30 chairs, 2 dining tables, 1 tv, 2 laptops, 60.8ms
Speed: 4.7ms preprocess, 60.8ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
Erro: 'NoneType' object has no attribute 'int'

0: 384x640 36 chairs, 3 dining tables, 1 tv, 1 laptop, 64.0ms
Speed: 4.0ms preprocess, 64.0ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)
Erro: 'NoneType' object has no attribute 'int'

0: 384x640 2 chairs, 64.4ms
Speed: 1.5ms preprocess, 64.4ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 chairs, 55.0ms
Speed: 1.6ms preprocess, 55.0ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 chairs, 54.5ms
Speed: 1.4ms preprocess, 54.5ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 chairs, 64.8ms
Speed: 1.7ms preprocess, 64.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 chairs, 66.5ms
Speed: 1.0ms preprocess, 66.5ms inference, 0.9ms pos


0: 384x640 2 chairs, 48.7ms
Speed: 1.4ms preprocess, 48.7ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 chairs, 54.5ms
Speed: 1.3ms preprocess, 54.5ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 chairs, 62.8ms
Speed: 1.3ms preprocess, 62.8ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 chairs, 51.7ms
Speed: 1.7ms preprocess, 51.7ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 chairs, 50.3ms
Speed: 5.1ms preprocess, 50.3ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 chairs, 51.7ms
Speed: 1.3ms preprocess, 51.7ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 chairs, 51.1ms
Speed: 1.3ms preprocess, 51.1ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 chairs, 52.1ms
Speed: 1.2ms preprocess, 52.1ms inference, 0.7ms postprocess per image at shape (1, 3, 38


0: 384x640 1 person, 2 chairs, 48.4ms
Speed: 5.2ms preprocess, 48.4ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 chairs, 48.0ms
Speed: 3.1ms preprocess, 48.0ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 chairs, 48.1ms
Speed: 1.4ms preprocess, 48.1ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 chairs, 50.8ms
Speed: 1.1ms preprocess, 50.8ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 chairs, 46.5ms
Speed: 2.3ms preprocess, 46.5ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 chairs, 50.7ms
Speed: 1.5ms preprocess, 50.7ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 chairs, 50.4ms
Speed: 1.4ms preprocess, 50.4ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 chairs, 49.9ms
Speed: 1.

Speed: 1.8ms preprocess, 49.0ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 chairs, 48.2ms
Speed: 1.5ms preprocess, 48.2ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 chairs, 47.0ms
Speed: 1.2ms preprocess, 47.0ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 chairs, 49.8ms
Speed: 1.4ms preprocess, 49.8ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 chairs, 48.5ms
Speed: 2.1ms preprocess, 48.5ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 chairs, 52.5ms
Speed: 1.4ms preprocess, 52.5ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 chairs, 51.3ms
Speed: 2.2ms preprocess, 51.3ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 chairs, 56.5ms
Speed: 1.0ms preprocess, 56.5ms inference, 0.8ms


0: 384x640 1 person, 2 chairs, 49.9ms
Speed: 1.4ms preprocess, 49.9ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 chairs, 49.0ms
Speed: 2.9ms preprocess, 49.0ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 chairs, 46.9ms
Speed: 2.1ms preprocess, 46.9ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 chairs, 48.9ms
Speed: 1.4ms preprocess, 48.9ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 chairs, 49.8ms
Speed: 6.8ms preprocess, 49.8ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 chairs, 47.2ms
Speed: 1.2ms preprocess, 47.2ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 chairs, 50.1ms
Speed: 1.2ms preprocess, 50.1ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 chairs, 48.4ms
Speed: 1.

Speed: 2.6ms preprocess, 51.8ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 chairs, 51.4ms
Speed: 1.5ms preprocess, 51.4ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 chairs, 51.3ms
Speed: 1.2ms preprocess, 51.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 chairs, 47.8ms
Speed: 1.0ms preprocess, 47.8ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 chairs, 50.0ms
Speed: 1.2ms preprocess, 50.0ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 chairs, 47.0ms
Speed: 1.2ms preprocess, 47.0ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 chairs, 52.1ms
Speed: 1.1ms preprocess, 52.1ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 chairs, 53.7ms
Speed: 1.1ms preprocess, 53.7ms inference, 0.7ms


0: 384x640 2 persons, 1 chair, 1 tv, 52.9ms
Speed: 1.2ms preprocess, 52.9ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 1 chair, 1 tv, 78.4ms
Speed: 6.2ms preprocess, 78.4ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 1 chair, 1 tv, 68.6ms
Speed: 6.5ms preprocess, 68.6ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 1 chair, 1 tv, 66.8ms
Speed: 2.3ms preprocess, 66.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 1 chair, 1 tv, 59.6ms
Speed: 1.7ms preprocess, 59.6ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 1 chair, 1 tv, 49.8ms
Speed: 2.4ms preprocess, 49.8ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 1 chair, 1 tv, 50.6ms
Speed: 1.2ms preprocess, 50.6ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 38


0: 384x640 3 persons, 1 chair, 1 tv, 55.1ms
Speed: 1.2ms preprocess, 55.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 1 chair, 1 tv, 55.2ms
Speed: 2.3ms preprocess, 55.2ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 1 chair, 1 tv, 53.8ms
Speed: 5.6ms preprocess, 53.8ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 1 chair, 1 tv, 54.7ms
Speed: 3.5ms preprocess, 54.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 1 chair, 1 tv, 54.6ms
Speed: 2.3ms preprocess, 54.6ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 1 chair, 1 tv, 55.5ms
Speed: 1.4ms preprocess, 55.5ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 1 chair, 1 tv, 53.8ms
Speed: 1.5ms preprocess, 53.8ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 38


0: 384x640 2 persons, 2 chairs, 1 tv, 52.1ms
Speed: 1.0ms preprocess, 52.1ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 2 chairs, 1 tv, 47.5ms
Speed: 4.4ms preprocess, 47.5ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 2 chairs, 1 tv, 47.6ms
Speed: 1.0ms preprocess, 47.6ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 2 chairs, 1 tv, 51.4ms
Speed: 2.6ms preprocess, 51.4ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 2 chairs, 1 tv, 54.6ms
Speed: 1.8ms preprocess, 54.6ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 2 chairs, 1 tv, 52.7ms
Speed: 1.1ms preprocess, 52.7ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 2 chairs, 1 tv, 51.5ms
Speed: 1.0ms preprocess, 51.5ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

Speed: 1.6ms preprocess, 48.5ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 2 chairs, 1 tv, 50.0ms
Speed: 1.9ms preprocess, 50.0ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 2 chairs, 1 tv, 47.3ms
Speed: 1.4ms preprocess, 47.3ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 2 chairs, 1 tv, 49.3ms
Speed: 1.7ms preprocess, 49.3ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 2 chairs, 1 tv, 49.5ms
Speed: 1.4ms preprocess, 49.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 2 chairs, 1 tv, 47.8ms
Speed: 1.3ms preprocess, 47.8ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 2 chairs, 1 tv, 47.6ms
Speed: 5.7ms preprocess, 47.6ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 2 chairs, 1 tv, 52.3ms